In [ ]:
import matplotlib.pyplot as plt
import itertools
from datetime import datetime, timedelta
import csv
import pandas as pd
import datetime as dt
import pandas as pd
import numpy as np
import networkx as nx
import spacy
nlp = spacy.load("en_core_web_lg")

In [2]:
df0 = pd.read_csv('Dataset_WorldCup.csv', lineterminator='\n')
df0.rename(columns = {'Name':'Topic','without_stopwords':'body'}, inplace = True)
df0.drop(columns='Unnamed: 0', axis=1, inplace=True)
df0['created_utc'] = pd.to_datetime(df0['created_utc'], errors='coerce')
df0 = df0[df0['created_utc'].notnull()]

In [4]:
data_iniziale = '2022-12-09 00:00:00' #'2022-11-01 00:00:00'
data_finale = '2022-12-31 23:59:59' #'2022-12-31 23:59:52'
data_iniziale=dt.datetime.strptime(data_iniziale, '%Y-%m-%d %H:%M:%S')
data_finale=dt.datetime.strptime(data_finale, '%Y-%m-%d %H:%M:%S')
intervallo = dt.timedelta(days=1)
data_attuale = data_iniziale

In [5]:
def Convert(string):
    
    string = string.replace(",", "")
    string = string.replace("[", "")
    string = string.replace("]", "")
    string = string.replace("'", "")
    li = list(string.split(" "))
    return li

df0['Topic']=df0['Topic'].apply(lambda x: Convert(x))




In [ ]:
while data_attuale <= data_finale:

    G = nx.Graph()

    intervallo_inizio = data_attuale
    intervallo_fine = data_attuale + intervallo
    selezione = (df0['created_utc'] > intervallo_inizio) & (df0['created_utc'] < intervallo_fine)
    df = df0.loc[selezione]
    
    lista = []
    print(len(df['body'].values))
    for i,e in enumerate(df['body'].values):
        print(i)
        lista.append(nlp(str(e)))
    df['nlp'] = lista
    
    
    print(data_attuale)

    ids = 'author' 
    G.add_nodes_from(df[ids].values, color='red')
    
    comments = 'name' 
    G.add_nodes_from(df[comments].values, color='blue')

    topics = 'Topic'
    topic_list = df[topics]
    topic_flat = list(set(itertools.chain(*topic_list)))
    G.add_nodes_from(topic_flat, color='green')

    for index, row in topic_similarity.iterrows():
        topic1= row['Topic1']
        topic2= row['Topic2']
        if (topic1 in topic_flat) and (topic2 in topic_flat):
            sim = row['Similarity']
            if sim > 0:
                G.add_edge(topic1, topic2, weight=sim)
    print('step0')

    for index, row in df.iterrows():
        source = row['name']
        target = row['parent_id']
        try:
            comm1 = row['nlp']
            comm2 = df.loc[df['name']==row['parent_id']]['nlp']
            sim_comm = comm1.similarity(comm2)
            
            G.add_edge(source, target, weight=sim_comm)
        except:
            G.add_edge(source, target, weight=0)
    
    parent_id = set(df['parent_id'])
    for  idx, id1 in enumerate(parent_id):
        
        fratelli = df.loc[df['parent_id']==id1]
        l = len(fratelli)
        print(idx,l)
        for i in range(l):
            source= fratelli.iloc[i]['name']
            comm1 = fratelli.iloc[i]['nlp']
            for k in range(l):
                if i<k:

                    comm2 = fratelli.iloc[k]['nlp']
                    target= fratelli.iloc[k]['name']
                    sim_comm = comm1.similarity(comm2)

                    G.add_edge(source, target, weight=sim_comm)         
            
            
    
    
    for index, row in df.iterrows():
        source = row['author']
        target = df.loc[df['name']==row['parent_id']]['author'].values
        if len(target)>0:
            if G.has_edge(source, target[0]):
                G[source][target[0]]['weight'] += 1 
            else:
                G.add_edge(source, target[0], weight=1)

    for index, row in df.iterrows(): 
        score = row['score']
        source = row['author']
        target = row['name']
        G.add_edge(source, target, weight=score)
  
    
    for index, row in df.iterrows(): 
        source = row['name']
        target = row['Topic']
        for t in target:
            G.add_edge(source, t) 

    nx.write_gexf(G, ('Graphs/'+str(data_attuale)+'.gexf'),encoding='utf-8', prettyprint=True, version='1.2draft')
    data_attuale += intervallo
    
